In [1]:
from matplotlib.pylab import*

from scipy.interpolate import griddata
from matplotlib import cm
import pandas as pd
from time import perf_counter
from sympy import*
import warnings
warnings.filterwarnings('ignore')

from Inputs import*
    
order = array([2,3])        # polynomial order
Nv = array([8,16,24,32])
kstages = 3
cfl = 0.25
dt = 1e-2
Tfinal = 0.1
iplot = False            # plot the solution
iconverg = False

time_method = "BDF2"      # IRK, BDF2 or BDF3
integration_type = 1      # % = 1 is inexact and = 2 is exact
icase = 2                 # select icase: 1,2,3,4

alpha = 1              
beta = 0#-1/2                  # Dirichlet: alpha = 0, beta = 1
                               # Neumann: alpha = 1, beta = 0
                               # Robin: alpha = 1, beta != 0

x_boundary = [5,5]    # Bottom and Top (x = -1 and x = +1)
y_boundary = [5,5]    # Left and Right (y = -1 and x = +1)

Visualisation2(order,Nv,time_method,kstages,integration_type,icase,Tfinal,alpha,beta,x_boundary,y_boundary)

Problem: Diffusion
Domain: [0, 6.283185307179586]
Diffusivity: 1.0
Boundary conditions:  Dirichlet & 

N = 2, nel = 64, Np = 289
	dt = 2.7778e-03
	Number of time steps = 36
	l2_norm: 0.003744178632815008
	walltime: 0.19671786299999994
N = 2, nel = 256, Np = 1089
	dt = 1.9960e-04
	Number of time steps = 501
	l2_norm: 0.00027266948879348357
	walltime: 1.6443059900000003
N = 2, nel = 576, Np = 2401
	dt = 4.1085e-05
	Number of time steps = 2434
	l2_norm: 5.641110443143319e-05
	walltime: 18.040335238
N = 2, nel = 1024, Np = 4225
	dt = 1.3268e-05
	Number of time steps = 7537
	l2_norm: 1.8263920481026844e-05
	walltime: 67.726844255
N = 3, nel = 64, Np = 625
	dt = 6.0606e-04
	Number of time steps = 165
	l2_norm: 0.0008441716234276927
	walltime: 0.8006788010000037
N = 3, nel = 256, Np = 2401
	dt = 4.1085e-05
	Number of time steps = 2434
	l2_norm: 5.776734636800124e-05
	walltime: 9.738727733999994
N = 3, nel = 576, Np = 5329
	dt = 8.3410e-06
	Number of time steps = 11989
	l2_norm: 1.176331539771

In [2]:
import pandas 

idx = pandas.IndexSlice

# Integration  : 
# time steps   : Number of time steps to time tfinal
# dt           : Time step size
# CFL          : CFL number
# Errors       : (1-norm, 2-norm and inf-norm) errors

ex_list = ['Diffusion']

cols = ['time steps', 'cfl','2-norm']
elements = Nv
iterables = [ex_list, ['CG'],['exact','inexact'], order, elements]

index = pandas.MultiIndex.from_product(iterables,names=['Eq.','method','integration','order','elements'])
df = pandas.DataFrame(index=index,columns=cols).sort_index()

df.loc[idx['Diffusion','CG',['inexact']]]

time steps  cfl 2-norm
Eq.       method integration order elements                       
Diffusion CG     inexact     2     8               NaN  NaN    NaN
                                   16              NaN  NaN    NaN
                                   24              NaN  NaN    NaN
                                   32              NaN  NaN    NaN
                             3     8               NaN  NaN    NaN
                                   16              NaN  NaN    NaN
                                   24              NaN  NaN    NaN
                                   32              NaN  NaN    NaN

In [3]:
output_file = 'output.dat'

# Open file as a binary file
fout = open(output_file,"rb")  
nlen = fromfile(fout,dtype=int,count=1)

# Create a data type for storing results;  use 'd' for double precision entries
dt_data = dtype([('N',int),('M',int),('cfl','d'),('2-norm','d')])

# Create a nested data type
dt_info = dtype([('order',int),('integType',int)])
dt_block = dtype([('info',dt_info),('data',(dt_data,nlen))])

# Read in four sets of data (orders 1,2 and integType 1,2)
out_data = fromfile(fout,dtype=dt_block, count=2)
fout.close()

# Store data in DataFrame
for k in range(2):                  # order = (1,2) x integType = (1,2)
    order = out_data[k]['info']['order']
    Itype = out_data[k]['info']['integType']
    if Itype == 1:        
        typex = 'inexact'
    else:
        typex = 'exact'
    
    for d in out_data[k]['data']:
        N = d[0]
        a = [d[i] for i in range(1,len(d))]
        df.loc[idx['Diffusion','CG',typex,order,N],cols] = a


# Format the data when displaying the DataFrame

fstr = {'cfl' : "{:.2f}".format, 
        '2-norm' : '{:.4e}'.format,}
        
df_out = df.loc[idx['Diffusion','CG',['inexact'],:],:]
df_out.style.format(fstr)